In [52]:
class tree():
            
    def __init__(self, branch_len=None, seq_leaf=None,parent=None,child=None, nuc_list=None,lkl_list=[]):
        self.seq_leaf = seq_leaf
        self.child = child
        self.parent=parent
        self.branch_length = branch_len
        self.nuc_list=nuc_list
        self.lkl_list=lkl_list
    def __len__(self):
        return len(self.seq_leaf)
    
    def getSequence(self):
        return self.seq_leaf
    
    def getBranchLength(self):
        return self.branch_length
       
    def getChildrenNodes(self):
        return self.child
    
    def get_nuc_list(self):
        return self.nuc_list
    
    def get_lkl_list(self):
        return self.lkl_list
    
    def initialize_nuc_list(self):
        if self.seq_leaf:
            bases={"A":[1,0,0,0],
                "C":[0,1,0,0],
                "G":[0,0,1,0],
                "T":[0,0,0,1]}
            self.nuc_list=list()
            for base in self.seq_leaf:
                self.nuc_list.append(bases[base])
            return self.nuc_list
        else: return None
    



In [53]:
branch_lengths_path="data/branchlength.dat"
tree_path="data/table.dat"
msa_path="data/msa.dat"



with open(tree_path,"r") as tree_file, open(branch_lengths_path,"r") as BL_file, open(msa_path,"r") as msa_file:
    node_added_tmp=[] ## list with nodes added to trr to upadte children
    Tree_Data=[]
    line_num=0
    bl=BL_file.readline().split(",")
    seq_for_leaves = {}
    for line in msa_file:
       (key, val) = line.split()
       seq_for_leaves[int(key)] = val


    for parent_child in tree_file:
        tmp_list=parent_child.split(",")
        parent_node_name=tmp_list[0]
 
        child_node_name=tmp_list[1].rstrip()
        branch_len=bl[line_num]
        if int(child_node_name) in seq_for_leaves.keys():
            leaf_seq=seq_for_leaves[int(child_node_name)]
        else:
            leaf_seq=None
        new_node = tree(branch_len=branch_len,
            seq_leaf=leaf_seq,
            parent=parent_node_name,
            child=child_node_name,
            nuc_list=[])
        
        
        Tree_Data.append(new_node)
        line_num+=1
    


    


In [54]:
for obj in Tree_Data:
    print("Parent name  : ",obj.parent)
    print("Child name   : ",obj.child)
    print("Branch length: ",obj.branch_length)
    print("sequence     : ", obj.seq_leaf)
    print("nuc list     : ", obj.initialize_nuc_list())
    print("nuc_list updated: " , obj.get_nuc_list())
    print("lkl_list updated: " , obj.get_lkl_list())


Parent name  :  9
Child name   :  1
Branch length:  0.1
sequence     :  AGATCAAGATCAAGATCAAGATCAAGATCA
nuc list     :  [[1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0]]
nuc_list updated:  [[1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0]]
lkl_list updated:  [

In [ ]:

import numpy as np
import scipy
from scipy.linalg import expm, sinm, cosm
Q=np.array([[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]])


#MAIN PROGRAMME
#we should have branch lengths, sequences (not all nodes ahve sequences--> only the leaf nodes) , tree in our class data structure

# this dict will store each node and the probabilities of 4 nucleotides at each position
# dict = {key is node name: values are list of lists}
#lkl_dict =  {node_name:nuc_likelihood}  
lkl_dict=dict() 

for node in Tree_Data:
    if node.getSequence() is not None: # if it is not internal node 
        nuc_list=node.get_nuc_list() # which nuc is present at each pposition --> a list of lists like [[1,0,0,0],[0,1,0,0]....]
        Probability_Matrix = scipy.linalg.expm(Q * float(node.getBranchLength())) # same for all positions
        for pos_list in nuc_list: # for each pos we have a list like [1,0,0,0] in nuc_list
            nuc_lkl = numpy.matmul(Probability_Matrix, pos_list ) # calculate liklihood of each nucleotide
            merge nuc_lkl to lkl_dict[node] # add to dict
   
    # if it is an internal node:
    else:
        child_names=node.get_children() # get_children will return a list of child node names --> two children per parent
        childA_lkl = lkl_dict[child_names[0]] # we will get a list of lists with prob for each nuclleotide at each position
        childB_lkl = lkl_dict[child_names[1]]
        for pos_list  in range(len(childA_lkl)): #using all positions in both children nodes, calculate ancetral nucleotide liklihoods 
            joint_lkl_AB = [childA_lkl[pos_list ][i] * childB_lkl[pos_list ][i] for i in range(4)]
            anc_lkl= numpy.matmul(joint_lkl_AB, [0.25,0.25,0.25,0.25] ) # all nucleotides ahve sam eprobabilities
            merge anc_lkl to lkl_dict[node]
return sum(math.log(anc_lkl)) # values in npmm will be the ones for root

In [55]:
import numpy as np
import scipy
from scipy.linalg import expm, sinm, cosm
Q=np.array([[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]])


for node in Tree_Data:
    if node.getSequence() is not None: # if it is not internal node 
        nuc_list=node.get_nuc_list() # which nuc is present at each pposition --> a list of lists like [[1,0,0,0],[0,1,0,0]....]
        Probability_Matrix = scipy.linalg.expm(Q * float(node.getBranchLength())) # same for all positions
        #print(Probability_Matrix)
        for pos_list in nuc_list:
            node.lkl_list.append(np.matmul(Probability_Matrix,pos_list))
            #print(node.get_lkl_list())


for node in Tree_Data:
    print("Parent name  : ",node.parent)
    print("Child name   : ",node.child)
    print("sequence     : ", node.seq_leaf)
    print("lkl_list updated: " , node.get_lkl_list())

'''


for childA in Tree_Data:
    for childB in Tree_Data:
        if childA.get_sequence() == None and childA.get_sequence() == None
        if (childA.parent == childB.parent) and (childA.child!=childB.child):
            print("reached step 0: child names, A:", childA.child,"B:",childB.child  )
            print(childB.lkl_list)
            break
            if len(childB.get_lkl_list()) == 0 and len(childA.get_lkl_list()) ==0:
                childA.lkl_list.append(1)
                childB.lkl_list.append(1)
                print(childB.get_lkl_list())
                print(childA.parent)
            

'''


'''
for node in lkl_dict:
    print("dict", node ," ",lkl_dict[node])
'''

Parent name  :  9
Child name   :  1
sequence     :  AGATCAAGATCAAGATCAAGATCAAGATCA
lkl_list updated:  [array([1.12295617, 0.12295617, 0.12295617, 0.12295617]), array([0.12295617, 0.12295617, 1.12295617, 0.12295617]), array([1.12295617, 0.12295617, 0.12295617, 0.12295617]), array([0.12295617, 0.12295617, 0.12295617, 1.12295617]), array([0.12295617, 1.12295617, 0.12295617, 0.12295617]), array([1.12295617, 0.12295617, 0.12295617, 0.12295617]), array([1.12295617, 0.12295617, 0.12295617, 0.12295617]), array([0.12295617, 0.12295617, 1.12295617, 0.12295617]), array([1.12295617, 0.12295617, 0.12295617, 0.12295617]), array([0.12295617, 0.12295617, 0.12295617, 1.12295617]), array([0.12295617, 1.12295617, 0.12295617, 0.12295617]), array([1.12295617, 0.12295617, 0.12295617, 0.12295617]), array([1.12295617, 0.12295617, 0.12295617, 0.12295617]), array([0.12295617, 0.12295617, 1.12295617, 0.12295617]), array([1.12295617, 0.12295617, 0.12295617, 0.12295617]), array([0.12295617, 0.12295617, 0.12295617,

'\nfor node in lkl_dict:\n    print("dict", node ," ",lkl_dict[node])\n'

In [35]:
childA.lkl_list

[array([1.12295617, 0.12295617, 0.12295617, 0.12295617]),
 array([0.12295617, 0.12295617, 1.12295617, 0.12295617]),
 array([1.12295617, 0.12295617, 0.12295617, 0.12295617]),
 array([0.12295617, 0.12295617, 0.12295617, 1.12295617]),
 array([0.12295617, 1.12295617, 0.12295617, 0.12295617]),
 array([1.12295617, 0.12295617, 0.12295617, 0.12295617]),
 array([1.12295617, 0.12295617, 0.12295617, 0.12295617]),
 array([0.12295617, 0.12295617, 1.12295617, 0.12295617]),
 array([1.12295617, 0.12295617, 0.12295617, 0.12295617]),
 array([0.12295617, 0.12295617, 0.12295617, 1.12295617]),
 array([0.12295617, 1.12295617, 0.12295617, 0.12295617]),
 array([1.12295617, 0.12295617, 0.12295617, 0.12295617]),
 array([1.12295617, 0.12295617, 0.12295617, 0.12295617]),
 array([0.12295617, 0.12295617, 1.12295617, 0.12295617]),
 array([1.12295617, 0.12295617, 0.12295617, 0.12295617]),
 array([0.12295617, 0.12295617, 0.12295617, 1.12295617]),
 array([0.12295617, 1.12295617, 0.12295617, 0.12295617]),
 array([1.1229

In [47]:
class Tree():
    def __init__(self, branch_len=None, seq_leaf=None, parent=None, child=None, nuc_list=None):
        self.seq_leaf = seq_leaf
        self.child = child
        self.parent = parent
        self.branch_length = branch_len
        self.nuc_list = nuc_list

    def __len__(self):
        return len(self.seq_leaf)

    def getSequence(self):
        return self.seq_leaf

    def getBranchLength(self):
        return self.branch_length

    def getChildrenNodes(self):
        return self.child

    def get_nuc_list(self):
        return self.nuc_list

    def initialize_nuc_list(self):
        if self.seq_leaf:
            bases = {
                "A": [1, 0, 0, 0],
                "C": [0, 1, 0, 0],
                "G": [0, 0, 1, 0],
                "T": [0, 0, 0, 1]
            }
            self.nuc_list = list()
            for base in self.seq_leaf:
                self.nuc_list.append(bases[base])
            return self.nuc_list
        else:
            return None

    def updateAttributeByName(self, attribute_name, new_value):
        if hasattr(self, attribute_name):
            setattr(self, attribute_name, new_value)
            print(f"Updated attribute '{attribute_name}' to '{new_value}'")
        else:
            print(f"Attribute '{attribute_name}' does not exist in this instance")


# Example usage:
my_tree = Tree(branch_len=5, seq_leaf="ACTG", parent=None, child=None, nuc_list=None)
print(f"Branch Length before update: {my_tree.getBranchLength()}")

# Update branch length by name
my_tree.updateAttributeByName("branch_length", 10)
print(f"Branch Length after update: {my_tree.getBranchLength()}")


Branch Length before update: 5
Updated attribute 'branch_length' to '10'
Branch Length after update: 10
